# 10.1 그룹 연산에 대한 고찰

그룹 연산은 Series, DataFrame 같은 판다스 객체나 다른 객체에 들어 있는 데이터를 하나 이상의 키 기준으로 분리한다.

분리하고 나면 함수를 각 그룹에 적용시켜 새로운 값을 얻어낸다.

함수를 적용한 결과를 하나의 객체로 결합한다.

#### 각 그룹의 색인의 다양한 형태
- 그룹으로 묶을 축과 동일한 길이의 리스트나 배열
- DataFrame의 열 이름을 지정하는 값
- 그룹으로 묶을 값과 그룹 이름에 대응하는 딕셔너리나 Series 객체
- 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [11]:
df = pd.DataFrame({"key1": ["a", "a", None, "b", "b", "a", None],
                    "key2": pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                    "data1": np.random.standard_normal(7),
                    "data2": np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.508892,0.807953
1,a,2,-1.218085,0.189996
2,None,1,-0.620771,-0.048237
3,b,2,-0.170985,1.508500
4,b,1,0.864907,0.762616
5,a,<NA>,0.696528,0.981092
6,None,1,-0.944634,0.705794


grouped는 groupby의 객체이다. (df["data1"]으로 참조되는 중간값에 대한 것 외에는 계산X)

이 객체는 그룹 연산을 위해 필요한 모든 정보를 가지고 각 그룹에 연산을 적용할 수 있게 해준다.

In [12]:
grouped = df["data1"].groupby(df["key1"])
grouped

groupby 객체의 mean 메서드를 사용해서 그룹별 평균을 구할 수 있다.

In [13]:
grouped.mean()

key1
a   -0.343483
b    0.346961
Name: data1, dtype: float64

In [14]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.508892
      2      -1.218085
b     1       0.864907
      2      -0.170985
Name: data1, dtype: float64

데이터를 두 개의 색인으로 묶었고, 그 결과 계층적 색인을 갖는 Series를 얻었다.

In [15]:
means.unstack()

key2,1,2
key1,,
a,-0.508892,-1.218085
b,0.864907,-0.170985


In [16]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.343483,0.659680
b,1.5,0.346961,1.135558


In [17]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,-0.302348,0.557032
2,-0.694535,0.849248


In [18]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.508892  0.807953
     2    -1.218085  0.189996
b    1     0.864907  0.762616
     2    -0.170985  1.508500

In [19]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [20]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [21]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [22]:
# 크기를 구하는 함수 count
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


길이만 같으면 어떤 배열이라도 상관없다.

In [26]:
states = np.array(["OK", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

df["data1"].groupby([states, years]).mean()

CA  2005   -0.260779
    2006   -0.620771
OH  2005   -0.170985
    2006   -0.039864
OK  2005   -0.508892
Name: data1, dtype: float64

In [27]:
df["data1"].groupby([states, years]).mean().unstack()

,2005,2006
CA,-0.260779,-0.620771
OH,-0.170985,-0.039864
OK,-0.508892,NaN


In [28]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,-0.343483,0.659680
b,1.5,0.346961,1.135558


In [29]:
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,-0.302348,0.557032
2,-0.694535,0.849248


In [30]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.508892  0.807953
     2    -1.218085  0.189996
b    1     0.864907  0.762616
     2    -0.170985  1.508500

`size 메서드`는 그룹의 크기를 담고 있는 Series를 반환한다.

In [31]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

그룹 색인에서 누락된 값은 결과에서 제외되는데, `dropna=False 인수`를 넘겨서 비활성화할 수 있다.

In [32]:
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [33]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

`count`는 크기를 구하는 그룹 함수로, 각 그룹에서 널이 아닌 값의 개수를 계산한다.

In [34]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


## 10.1.1 그룹 간 순회하기

groupby 메서드에서 반환된 객체는 순회(이터레이션)를 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환한다.

In [14]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.585725  1.144628
1    a     2  0.944335 -0.459025
5    a  <NA> -0.830981  0.712074
b
  key1  key2     data1     data2
3    b     2  1.171966 -0.025023
4    b     1 -0.101640  1.002491


색인이 여러 개 존재하는 경우에는 튜플의 첫 번째 원소가 색인값이 된다.

In [15]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1  0.585725  1.144628
('a', 2)
  key1  key2     data1     data2
1    a     2  0.944335 -0.459025
('b', 1)
  key1  key2    data1     data2
4    b     1 -0.10164  1.002491
('b', 2)
  key1  key2     data1     data2
3    b     2  1.171966 -0.025023


한 줄이면 그룹별 데이터를 딕셔너리형으로 바꿔 유용하게 사용할 수 있다.

In [16]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["a"]

,key1,key2,data1,data2
0,a,1,0.585725,1.144628
1,a,2,0.944335,-0.459025
5,a,<NA>,-0.830981,0.712074


In [17]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,1.171966,-0.025023
4,b,1,-0.101640,1.002491


groupby 메서드는 axis="index"에 대해서 그룹을 만들지만, 다른 축으로 그룹을 만들 수 있다.

In [18]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0  0.585725  1.144628
1  0.944335 -0.459025
2  0.200174  0.121805
3  1.171966 -0.025023
4 -0.101640  1.002491
5 -0.830981  0.712074
6  0.835107 -0.107465
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


## 10.1.2 열이나 열의 일부만 선택하기

DataFrame에서 만든 GroupBy 객체를 열 이름이나 열 이름이 담긴 배열로 색인하면 수집을 위해 해당 열을 선택하게 된다.

In [35]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

대용량 데이터셋을 다룰 때 소수의 열만 집계하고 싶은 경우에 사용한다.

In [36]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.807953
     2     0.189996
b    1     0.762616
     2     1.508500

색인으로 얻은 객체는 groupby 메서드에 리스트나 배열을 넘겼을 경우 그룹으로 묶인 DataFrame이 되고, 단일 값으로 하나의 열 이름만 넘겼다면 그룹으로 묶인 Series 객체가 된다.

In [37]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

s_grouped.mean()

key1  key2
a     1       0.807953
      2       0.189996
b     1       0.762616
      2       1.508500
Name: data2, dtype: float64

## 10.2 데이터 집계

In [19]:
df

,key1,key2,data1,data2
0,a,1,0.585725,1.144628
1,a,2,0.944335,-0.459025
2,None,1,0.200174,0.121805
3,b,2,1.171966,-0.025023
4,b,1,-0.101640,1.002491
5,a,<NA>,-0.830981,0.712074
6,None,1,0.835107,-0.107465


In [20]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     5   -0.830981
      0    0.585725
b     4   -0.101640
      3    1.171966
Name: data1, dtype: float64

In [21]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.775316,1.603653
b,1,1.273605,1.027514


In [22]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.233026  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.535163  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.765030  0.944335   3.0  0.465892  0.829687 -0.459025  0.126524   
b     0.853564  1.171966   2.0  0.488734  0.726562 -0.025023  0.231856   

                                    
           50%       75%       max  
key1                                
a     0.712074  0.928351  1.144628  
b     0.488734  0.745613  1.002491  

[2 rows x 24 columns]

## 10.2.1 열에 여러 가지 함수 적용하기

In [23]:
tips = pd.read_csv("tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [24]:
tips["tip_pct"] = tips["tip"] / (tips["total_bill"] - tips["tip"])
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.063204
1,10.34,1.66,No,Sun,Dinner,3,0.191244
2,21.01,3.50,No,Sun,Dinner,3,0.199886
3,23.68,3.31,No,Sun,Dinner,2,0.162494
4,24.59,3.61,No,Sun,Dinner,4,0.172069


In [25]:
grouped = tips.groupby(["day", "smoker"])

grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")    # grouped_pct.agg(np.mean)

day   smoker
Fri   No        0.179740
      Yes       0.216293
Sat   No        0.190412
      Yes       0.179833
Sun   No        0.193617
      Yes       0.322021
Thur  No        0.193424
      Yes       0.198508
Name: tip_pct, dtype: float64

In [26]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.179740  0.039458      0.094263
     Yes     0.216293  0.077530      0.242219
Sat  No      0.190412  0.058626      0.352192
     Yes     0.179833  0.089496      0.446137
Sun  No      0.193617  0.060302      0.274897
     Yes     0.322021  0.538061      2.382107
Thur No      0.193424  0.056065      0.284273
     Yes     0.198508  0.057170      0.219047

In [27]:
grouped_pct.agg([("평균", "mean"), ("표준편차", np.std)])

평균      표준편차
day  smoker                    
Fri  No      0.179740  0.039458
     Yes     0.216293  0.077530
Sat  No      0.190412  0.058626
     Yes     0.179833  0.089496
Sun  No      0.193617  0.060302
     Yes     0.322021  0.538061
Thur No      0.193424  0.056065
     Yes     0.198508  0.057170

In [28]:
for k, v in grouped["tip_pct"]:
    print(k)
    print(v)

('Fri', 'No')
91     0.184308
94     0.166667
99     0.136861
223    0.231125
Name: tip_pct, dtype: float64
('Fri', 'Yes')
90     0.115518
92     0.210526
93     0.357737
95     0.133465
96     0.171821
97     0.142450
98     0.166574
100    0.282486
101    0.242326
220    0.220884
221    0.350101
222    0.288288
224    0.133446
225    0.181554
226    0.247219
Name: tip_pct, dtype: float64
('Sat', 'No')
19     0.193642
20     0.294798
21     0.156784
22     0.164697
23     0.238065
24     0.191106
25     0.151261
26     0.175901
27     0.187091
28     0.247126
29     0.180180
30     0.179012
31     0.157729
32     0.248756
33     0.134320
34     0.225362
35     0.175953
36     0.139762
37     0.221501
38     0.141026
39     0.190331
40     0.162319
57     0.060217
59     0.162013
64     0.176589
65     0.186060
66     0.176681
68     0.110318
70     0.196020
71     0.213220
74     0.175579
75     0.134989
104    0.242280
108    0.259669
110    0.272727
111    0.160000
212    0.228833
2

In [29]:
grouped["tip_pct"].agg(["count", "mean", "max"])

count      mean       max
day  smoker                           
Fri  No          4  0.179740  0.231125
     Yes        15  0.216293  0.357737
Sat  No         45  0.190412  0.412409
     Yes        42  0.179833  0.483092
Sun  No         57  0.193617  0.338101
     Yes        19  0.322021  2.452381
Thur No         45  0.193424  0.362976
     Yes        17  0.198508  0.317965

In [30]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.179740  0.231125          4  18.420000  22.75
     Yes         15  0.216293  0.357737         15  16.813333  40.17
Sat  No          45  0.190412  0.412409         45  19.661778  48.33
     Yes         42  0.179833  0.483092         42  21.276667  50.81
Sun  No          57  0.193617  0.338101         57  20.506667  48.17
     Yes         19  0.322021  2.452381         19  24.120000  45.35
Thur No          45  0.193424  0.362976         45  17.113111  41.19
     Yes         17  0.198508  0.317965         17  19.190588  43.11

In [31]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.179740  0.231125
     Yes        15  0.216293  0.357737
Sat  No         45  0.190412  0.412409
     Yes        42  0.179833  0.483092
Sun  No         57  0.193617  0.338101
     Yes        19  0.322021  2.452381
Thur No         45  0.193424  0.362976
     Yes        17  0.198508  0.317965

In [32]:
result["total_bill"]

count       mean    max
day  smoker                         
Fri  No          4  18.420000  22.75
     Yes        15  16.813333  40.17
Sat  No         45  19.661778  48.33
     Yes        42  21.276667  50.81
Sun  No         57  20.506667  48.17
     Yes        19  24.120000  45.35
Thur No         45  17.113111  41.19
     Yes        17  19.190588  43.11

In [33]:
pd.concat([result["tip_pct"], result["total_bill"]], axis='columns')

count      mean       max  count       mean    max
day  smoker                                                    
Fri  No          4  0.179740  0.231125      4  18.420000  22.75
     Yes        15  0.216293  0.357737     15  16.813333  40.17
Sat  No         45  0.190412  0.412409     45  19.661778  48.33
     Yes        42  0.179833  0.483092     42  21.276667  50.81
Sun  No         57  0.193617  0.338101     57  20.506667  48.17
     Yes        19  0.322021  2.452381     19  24.120000  45.35
Thur No         45  0.193424  0.362976     45  17.113111  41.19
     Yes        17  0.198508  0.317965     17  19.190588  43.11

In [34]:
ftuples = [("평균", "mean"), ("분산", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
                   평균        분산         평균          분산
day  smoker                                           
Fri  No      0.179740  0.001557  18.420000   25.596333
     Yes     0.216293  0.006011  16.813333   82.562438
Sat  No      0.190412  0.003437  19.661778   79.908965
     Yes     0.179833  0.008010  21.276667  101.387535
Sun  No      0.193617  0.003636  20.506667   66.099980
     Yes     0.322021  0.289509  24.120000  109.046044
Thur No      0.193424  0.003143  17.113111   59.625081
     Yes     0.198508  0.003268  19.190588   69.808518

In [35]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [36]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"], "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.136861  0.231125  0.179740  0.039458    9
     Yes     0.115518  0.357737  0.216293  0.077530   31
Sat  No      0.060217  0.412409  0.190412  0.058626  115
     Yes     0.036955  0.483092  0.179833  0.089496  104
Sun  No      0.063204  0.338101  0.193617  0.060302  167
     Yes     0.070274  2.452381  0.322021  0.538061   49
Thur No      0.078704  0.362976  0.193424  0.056065  112
     Yes     0.098918  0.317965  0.198508  0.057170   40

## 10.2.2 색인되지 않은 형태로 집계된 데이터 반환하기

as_index=False를 넘겨서 색인 작업을 비활성화할 수 있다.

In [37]:
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.179740
1,Fri,Yes,16.813333,2.714000,2.066667,0.216293
2,Sat,No,19.661778,3.102889,2.555556,0.190412
3,Sat,Yes,21.276667,2.875476,2.476190,0.179833
4,Sun,No,20.506667,3.167895,2.929825,0.193617
5,Sun,Yes,24.120000,3.516842,2.578947,0.322021
6,Thur,No,17.113111,2.673778,2.488889,0.193424
7,Thur,Yes,19.190588,3.030000,2.352941,0.198508


# 10.3 apply 메서드: 일반적인 분리-적용-병합

apply 메서드는 객체를 여러 조각으로 나누고,전달된 함수를 각 조각에 일괄적으로 적용한 후 이를 다시 합친다.

In [39]:
# 특정 열에서 가장 큰 값을 갖는 행을 선택하는 함수
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,2.452381
178,9.60,4.00,Yes,Sun,Dinner,2,0.714286
67,3.07,1.00,Yes,Sat,Dinner,1,0.483092
232,11.61,3.39,No,Sat,Dinner,2,0.412409
183,23.17,6.50,Yes,Sun,Dinner,4,0.389922
109,14.31,4.00,Yes,Sat,Dinner,2,0.387973


나눠진 DataFrame의 각 부분에 top 함수가 모두 적용되었고, pandas.concat을 이용해 하나로 합쳐진 다음 그룹 이름이 붙었다.

In [40]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.412409
       149        7.51  2.00     No  Thur   Lunch     2  0.362976
       51        10.29  2.60     No   Sun  Dinner     2  0.338101
       185       20.69  5.00     No   Sun  Dinner     5  0.318674
       88        24.71  5.85     No  Thur   Lunch     2  0.310180
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  2.452381
       178        9.60  4.00    Yes   Sun  Dinner     2  0.714286
       67         3.07  1.00    Yes   Sat  Dinner     1  0.483092
       183       23.17  6.50    Yes   Sun  Dinner     4  0.389922
       109       14.31  4.00    Yes   Sat  Dinner     2  0.387973

apply 메서드에 넘길 함수가 추가적인 인수나 예약어를 받는다면 함수 이름 뒤에 붙여서 넘겨주면 된다.

In [42]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.166667
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.228833
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.115821
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.138160
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.133465
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.245038
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.083632
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.131199

In [43]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.192237,0.057665,0.060217,0.158622,0.184308,0.227015,0.412409
Yes,93.0,0.218176,0.254295,0.036955,0.119534,0.181818,0.242326,2.452381


In [44]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.192237
       Yes         0.218176
std    No          0.057665
       Yes         0.254295
min    No          0.060217
       Yes         0.036955
25%    No          0.158622
       Yes         0.119534
50%    No          0.184308
       Yes         0.181818
75%    No          0.227015
       Yes         0.242326
max    No          0.412409
       Yes         2.452381
dtype: float64

describe 같은 메서드를 호출하려면 groupby는 내부적으로 다음 단계를 수행한다.

In [45]:
def f(group):
    return group.describe()

grouped.apply(f)

total_bill       tip  size   tip_pct
0 count    4.000000  4.000000  4.00  4.000000
  mean    18.420000  2.812500  2.25  0.179740
  std      5.059282  0.898494  0.50  0.039458
  min     12.460000  1.500000  2.00  0.136861
  25%     15.100000  2.625000  2.00  0.159215
...             ...       ...   ...       ...
7 min     10.340000  2.000000  2.00  0.098918
  25%     13.510000  2.000000  2.00  0.173762
  50%     16.470000  2.560000  2.00  0.181818
  75%     19.810000  4.000000  2.00  0.241984
  max     43.110000  5.000000  4.00  0.317965

[64 rows x 4 columns]

## 10.3.1 그룹 키 생략하기

원본 객체의 각 조각에 대한 색인과 그룹 키가 계층적 색인으로 사용되는데, group_keys=False를 전달해 그룹 키를 생략할 수 있다.

In [47]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.412409
       149        7.51  2.00     No  Thur   Lunch     2  0.362976
       51        10.29  2.60     No   Sun  Dinner     2  0.338101
       185       20.69  5.00     No   Sun  Dinner     5  0.318674
       88        24.71  5.85     No  Thur   Lunch     2  0.310180
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  2.452381
       178        9.60  4.00    Yes   Sun  Dinner     2  0.714286
       67         3.07  1.00    Yes   Sat  Dinner     1  0.483092
       183       23.17  6.50    Yes   Sun  Dinner     4  0.389922
       109       14.31  4.00    Yes   Sat  Dinner     2  0.387973

In [48]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.412409
149,7.51,2.00,No,Thur,Lunch,2,0.362976
51,10.29,2.60,No,Sun,Dinner,2,0.338101
185,20.69,5.00,No,Sun,Dinner,5,0.318674
88,24.71,5.85,No,Thur,Lunch,2,0.310180
172,7.25,5.15,Yes,Sun,Dinner,2,2.452381
178,9.60,4.00,Yes,Sun,Dinner,2,0.714286
67,3.07,1.00,Yes,Sat,Dinner,1,0.483092
183,23.17,6.50,Yes,Sun,Dinner,4,0.389922
109,14.31,4.00,Yes,Sat,Dinner,2,0.387973


## 10.3.2 사분위수 분석과 버킷 분석

pandas.cut, pandas.qcut 메서드를 사용해서 선택 크기만큼 혹은 표본 사분위수에 따라 데이터를 나눌 수 있다.

groupby와 조합하면 데이터셋에 대한 사분위수 분석이나 버킷 분석을 쉽게 수행할 수 있다.

### 등간격 버킷

In [69]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,0.294085,-1.825768
1,-0.159815,0.107631
2,0.217538,-0.284954
3,1.021964,-1.542220
4,-1.008432,0.399485


In [70]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0     (-0.123, 1.363]
1    (-1.608, -0.123]
2     (-0.123, 1.363]
3     (-0.123, 1.363]
4    (-1.608, -0.123]
5     (-0.123, 1.363]
6     (-0.123, 1.363]
7     (-0.123, 1.363]
8     (-0.123, 1.363]
9     (-0.123, 1.363]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.1, -1.608] < (-1.608, -0.123] < (-0.123, 1.363] < (1.363, 2.849]]

In [71]:
# quartiles에 대한 그룹 통계
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
         "count": group.count(), "max": group.mean()})

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count
data1                                            
(-3.1, -1.608]   data1 -3.094115 -2.015766     54
                 data2 -1.765740  0.128391     54
(-1.608, -0.123] data1 -1.606773 -0.698877    401
                 data2 -3.410443  0.030940    401
(-0.123, 1.363]  data1 -0.122132  0.534637    459
                 data2 -2.683743  0.002016    459
(1.363, 2.849]   data1  1.366600  1.851518     86
                 data2 -2.652674  0.159008     86

In [72]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.1, -1.608]   -3.094115 -1.617586    54 -2.015766 -1.765740  2.909049   
(-1.608, -0.123] -1.606773 -0.133150   401 -0.698877 -3.410443  2.714192   
(-0.123, 1.363]  -0.122132  1.352626   459  0.534637 -2.683743  2.857119   
(1.363, 2.849]    1.366600  2.848563    86  1.851518 -2.652674  2.396343   

                                  
                 count      mean  
data1                             
(-3.1, -1.608]      54  0.128391  
(-1.608, -0.123]   401  0.030940  
(-0.123, 1.363]    459  0.002016  
(1.363, 2.849]      86  0.159008

In [73]:
grouped.agg(["min", "max", "count", np.mean]).stack()

data1       data2
data1                                         
(-3.1, -1.608]   min     -3.094115   -1.765740
                 max     -1.617586    2.909049
                 count   54.000000   54.000000
                 mean    -2.015766    0.128391
(-1.608, -0.123] min     -1.606773   -3.410443
                 max     -0.133150    2.714192
                 count  401.000000  401.000000
                 mean    -0.698877    0.030940
(-0.123, 1.363]  min     -0.122132   -2.683743
                 max      1.352626    2.857119
                 count  459.000000  459.000000
                 mean     0.534637    0.002016
(1.363, 2.849]   min      1.366600   -2.652674
                 max      2.848563    2.396343
                 count   86.000000   86.000000
                 mean     1.851518    0.159008

### 사분위수

In [74]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0                   (0.00377, 0.7]
1                (-0.647, 0.00377]
2                   (0.00377, 0.7]
3                     (0.7, 2.849]
4    (-3.0949999999999998, -0.647]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.0949999999999998, -0.647] < (-0.647, 0.00377] < (0.00377, 0.7] < (0.7, 2.849]]

In [75]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    2
1    1
2    2
3    3
4    0
Name: data1, dtype: int64

In [76]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count
data1                                 
0     data1 -3.094115 -1.251450    250
      data2 -3.031963  0.050876    250
1     data1 -0.645868 -0.315552    250
      data2 -3.410443  0.035736    250
2     data1  0.004448  0.339364    250
      data2 -2.683743  0.000724    250
3     data1  0.700753  1.289749    250
      data2 -2.652674  0.048425    250

## 10.3.3 그룹별 값으로 결측치 채우기

누락된 데이터를 정리할 때, dropna를 사용해서 데이터를 걸러내는 방법도 있고,

fillna 메서드를 사용해서 고정된 값이나 데이터에서 도출된 특정한 값으로 채우는 방법도 있다.

In [77]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1   -0.625833
2         NaN
3   -0.678583
4         NaN
5    0.647930
dtype: float64

In [79]:
# 결측치를 평균값으로 채워 넣음
s.fillna(s.mean())

0   -0.218829
1   -0.625833
2   -0.218829
3   -0.678583
4   -0.218829
5    0.647930
dtype: float64

In [81]:
states = ["Ohio", "New York", "Vermont", "Florida",
         "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East","East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio          0.007209
New York      0.798311
Vermont       0.235422
Florida      -1.152962
Oregon       -1.402562
Nevada       -0.526068
California    0.281895
Idaho         0.246616
dtype: float64

In [82]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio          0.007209
New York      0.798311
Vermont            NaN
Florida      -1.152962
Oregon       -1.402562
Nevada             NaN
California    0.281895
Idaho              NaN
dtype: float64

In [83]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [84]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [85]:
data.groupby(group_key).mean()

East   -0.115814
West   -0.560333
dtype: float64

In [92]:
# 그룹의 평균값으로 누락된 값 채우기
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key,group_keys=False).apply(fill_mean)

Ohio          0.007209
New York      0.798311
Vermont      -0.115814
Florida      -1.152962
Oregon       -1.402562
Nevada       -0.560333
California    0.281895
Idaho        -0.560333
dtype: float64

그룹별로 미리 정의된 다른 값을 채워 넣어야할 때는 name이라는 속성을 이용한다.

In [91]:
fill_values = {"East": 0.5, "West": -1}

def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key,group_keys=False).apply(fill_func)

Ohio          0.007209
New York      0.798311
Vermont       0.500000
Florida      -1.152962
Oregon       -1.402562
Nevada       -1.000000
California    0.281895
Idaho        -1.000000
dtype: float64

## 10.3.4 랜덤 표본과 순열

대용량의 데이터셋을 몬테카를로 방법이나 다른 애플리케이션에서 사용하기 위해 랜덤 표본을 뽑아보자.

### 트럼프 타드 덱 만들기
블랙잭 같은 카드 게임에서 사용하는 카드 이름과 값을 색인으로 하는 52장의 카드를 Series 객체로 준비하자. (에이스("A")는 1로 취급)

In [100]:
suits = ["H", "S", "C", "D"]    # 하트, 스페이드, 클럽, 다이아몬드
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]

cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index=cards)

In [94]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [98]:
# 카드 5장 뽑기
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

5D     5
7D     7
3D     3
KC    10
5C     5
dtype: int64

각 카드의 마지막 글자가 모양을 나타내므로 이를 이용해서 그룹을 나누고 apply를 사용하자.

In [101]:
# 각 모양별로 카드 2잡 뽑기
def get_suit(card):
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  QC    10
   5C     5
D  4D     4
   3D     3
H  3H     3
   4H     4
S  JS    10
   8S     8
dtype: int64

group_keys=False를 넘겨서 다른 모양의 색인을 제외하고 선택된 카드만 남겨보자.

In [97]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C     5
8C     8
QD    10
5D     5
8H     8
3H     3
QS    10
7S     7
dtype: int64

## 10.3.5 그룹 가중평균과 상관관계

groupby의 분리-적용-결합 패러다임에서 그룹 가중평균 작업 같은 DataFrame에서 열 간의 연산이나 두 Series 간의 연산은 매우 일상적이다.

In [102]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,-0.326066,0.184216
1,a,-0.457056,0.207371
2,a,1.654949,0.234737
3,a,0.238397,0.186892
4,b,-0.808136,0.575754
5,b,-0.069144,0.041887
6,b,0.626588,0.880174
7,b,0.286385,0.894952


In [104]:
grouped = df.groupby("category")

def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg)

category
a    0.342081
b    0.141938
dtype: float64

### 몇몇 주식과 S&P 500 지수

In [106]:
close_px = pd.read_csv("stock_px.csv", parse_dates=True, index_col=0)

close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [107]:
close_px.tail()

,AAPL,MSFT,XOM,SPX
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [108]:
# SPX 열과 다른 열의 상관관계를 계산하는 함수
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [114]:
rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [110]:
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [111]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])

by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

## 10.3.6 그룹별 선형 회귀

함수가 판다스 객체나 스칼라 값을 반환하기만 한다면 groupby를 조금 더 복잡한 그룹별 통계분석을 위해 사용할 수 있다.

계량경제 라이브러리인 statsmodels를 사용해서 regress 함수를 작성하고 각 데이터 묶음마다 최소제곱법(OLS)으로 회귀를 수행할 수 있다.

In [116]:
import statsmodels.api as sm

def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [117]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 그룹 변환과 래핑되지 않은 groupby

transform 메서드는 apply와 유사하지만 다음과 같은 제약 사항이 있다.
- 그룹 모양대로 브로드캐스팅할 스칼라 값 생성
- 입력 그룹과 동일한 모양의 객체 생성
- 입력을 변경하면 안됨

In [118]:
df = pd.DataFrame({'key': ["a", "b", "c"] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [119]:
# 키별 그룹 평균
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [121]:
def get_mean(group):
    return group.mean()

g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [122]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [123]:
def times_two(group):
    return group * 2

g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [124]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [126]:
def normalize(x):
    return (x - x.mean()) / x.std()

g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [127]:
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [128]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [130]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

# 10.5 피벗 테이블과 교차표

In [131]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.063204
1,10.34,1.66,No,Sun,Dinner,3,0.191244
2,21.01,3.50,No,Sun,Dinner,3,0.199886
3,23.68,3.31,No,Sun,Dinner,2,0.162494
4,24.59,3.61,No,Sun,Dinner,4,0.172069


In [132]:
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.179740   18.420000
     Yes     2.066667  2.714000  0.216293   16.813333
Sat  No      2.555556  3.102889  0.190412   19.661778
     Yes     2.476190  2.875476  0.179833   21.276667
Sun  No      2.929825  3.167895  0.193617   20.506667
     Yes     2.578947  3.516842  0.322021   24.120000
Thur No      2.488889  2.673778  0.193424   17.113111
     Yes     2.352941  3.030000  0.198508   19.190588

In [134]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.162612  0.202545
       Sat   2.555556  2.476190  0.190412  0.179833
       Sun   2.929825  2.578947  0.193617  0.322021
       Thur  2.000000       NaN  0.190114       NaN
Lunch  Fri   3.000000  1.833333  0.231125  0.236915
       Thur  2.500000  2.352941  0.193499  0.198508

In [138]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.162612  0.202545  0.192562
       Sat   2.555556  2.476190  2.517241  0.190412  0.179833  0.185305
       Sun   2.929825  2.578947  2.842105  0.193617  0.322021  0.225718
       Thur  2.000000       NaN  2.000000  0.190114       NaN  0.190114
Lunch  Fri   3.000000  1.833333  2.000000  0.231125  0.236915  0.236088
       Thur  2.500000  2.352941  2.459016  0.193499  0.198508  0.194895
All          2.668874  2.408602  2.569672  0.192237  0.218176  0.202123

In [139]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [141]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.160000  0.000000  0.000000
            Yes     0.000000  0.483092  0.000000  0.000000
       2    No      0.162612  0.198319  0.206535  0.190114
            Yes     0.211180  0.178877  0.400522  0.000000
       3    No      0.000000  0.183870  0.182962  0.000000
            Yes     0.000000  0.176599  0.183278  0.000000
       4    No      0.000000  0.177734  0.175289  0.000000
            Yes     0.133465  0.147074  0.254373  0.000000
       5    No      0.000000  0.000000  0.263344  0.000000
            Yes     0.000000  0.119284  0.070274  0.000000
       6    No      0.000000  0.000000  0.115821  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.222087
            Yes     0.288288  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.201503
            Yes     0.226641  0.000000  0.000000  0.191197
       3    No      0.231125  0.000000  0.000000  0.092162
            Yes     0.000000  0.000000  0.000000  0.257941
       4    No      0.000000  0.000000  0.000000  0.161573
            Yes     0.000000  0.000000  0.000000  0.186592
       5    No      0.000000  0.000000  0.000000  0.138160
       6    No      0.000000  0.000000  0.000000  0.211191

## 10.5.1 교차표

교차표는 그룹 빈도를 계산하는 특수한 피벗 테이블이다.

In [145]:
from io import StringIO

data = """Sample Nationality Handedness
1    USA    Right-handed
2    Japan    Left-handed
3    USA    Right-handed
4    Japan    Right-handed
5    Japan    Left-handed
6    Japan    Right-handed
7    USA    Right-handed
8    USA    Left-handed
9    Japan    Right-handed
10    USA    Right-handed"""

data = pd.read_table(StringIO(data), sep="\s+")
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [146]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [147]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244